In [58]:
import pandas as pd
from joblib import load
import numpy
import treelite
import treelite_runtime

In [8]:
data = numpy.load('data.npy')
week_data = pd.read_csv('week.csv')['CB_TOTAL'].to_numpy()
model = load('best.h5')
vacation = 0
week = 0

In [55]:
model = treelite.Model.load('model.bst', model_format='xgboost')

In [60]:
model.export_lib(toolchain='gcc', libpath='./mymodel.so', verbose=True)

[18:53:49] /workspace/src/compiler/ast_native.cc:44: Using ASTNativeCompiler
[18:53:49] /workspace/src/compiler/ast/split.cc:24: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.
[18:53:49] /workspace/src/c_api/c_api.cc:286: Code generation finished. Writing code to files...
[18:53:49] /workspace/src/c_api/c_api.cc:291: Writing file recipe.json...
[18:53:49] /workspace/src/c_api/c_api.cc:291: Writing file header.h...
[18:53:49] /workspace/src/c_api/c_api.cc:291: Writing file main.c...
[18:53:49] /home/bharris/.local/lib/python3.8/site-packages/treelite/contrib/util.py:104: Compiling sources files in directory ./tmpjm54k6fg into object files (*.o)...
[18:53:51] /home/bharris/.local/lib/python3.8/site-packages/treelite/contrib/util.py:133: Generating dynamic shared library ./tmpjm54k6fg/predictor.so...
[18:53:51] /home/bharris/.local/lib/python3.8/site-packages/treelite/contrib/__init__.py:278: Gen

In [61]:
model = treelite_runtime.Predictor('./mymodel.so', verbose=True)

[18:54:16] /home/bharris/.local/lib/python3.8/site-packages/treelite_runtime/predictor.py:309: Dynamic shared library /home/bharris/Research/Fire/RL/muzero-general/mymodel.so has been successfully loaded into memory


In [75]:
batch = treelite_runtime.Batch.from_npy2d(data[0].reshape(1,21))

In [79]:
predictor.predict(batch).round()

1.0

In [38]:
    def get_reward(week_data,week,vacation):
        try:
            callbacks = week_data[week]
            delta = 0
            reward = 0
            if callbacks >=0:
                delta = callbacks + vacation
                if delta <= 0:
                    #self.savings += callbacks*1000
                    reward = callbacks*1000
                elif vacation < 0:
                    reward = vacation*-1000
                else:
                    #self.savings += -1*self.vacation*1000  
                    reward = -1*vacation*1000
    #         else:
    #             delta = callbacks + self.vacation
    #             if delta >= 0:
    #                 self.vac += callbacks*-1
    #                 reward = callbacks*-1
    #             else:
    #                 self.vac += self.vacation
    #                 reward = self.vacation
    #         if delta <=0:
    #             reward = numpy.abs(self.min - delta)*10/numpy.abs(self.min)
    #         else:
    #             reward = numpy.abs(self.max - delta)*10/numpy.abs(self.max)
        except Exception as e: 
            print('error')
        return round(reward)
    
    def step(action,week,week_data):
        rewards =0
        try:
            vacation = 0
            if action < 6:
                vacation = action
            elif action > 5:
                vacation = (action - 5)*-1
            done = week == 51
            rewards = get_reward(week_data,week,vacation)
        except Exception as e: 
            rewards =-1000
            print(e)
        return rewards, done


In [44]:
step(3,51,week_data)

(-3000, True)